In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from itertools import groupby
from sklearn.model_selection import StratifiedKFold, KFold
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import mean_absolute_error as mae
import re


Bad key "text.kerning_factor" on line 4 in
d:\Users\96212\anaconda3\envs\ML\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


# 1.读取文件

In [3]:
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')

# 只显示数值特征
df_train.describe()

,ID,区域1,区域2,区域3,街道,房屋租金,服务费,供暖费用,电力基础价格,没有停车位,...,加热类型,房屋类型,邮政编码,房间数量,所处楼层,建筑楼层,最后翻新年份,建成年份,价格趋势,上传图片数
count,200000.000000,200000.00000,200000.000000,200000.000000,200000.000000,2.000000e+05,194878.000000,63507.000000,34806.000000,69183.000000,...,199881.000000,195799.000000,200000.000000,200000.000000,161736.000000,127329.000000,60166.000000,157522.000000,198679.00000,200000.000000
mean,99999.500000,5.11352,112.193435,1452.648445,8684.393950,7.050609e+02,150.694673,77.473713,89.115567,1.337482,...,2.383513,1.779018,1466.059120,2.638721,2.122106,3.573703,2013.928747,1966.374310,3.38789,9.789215
std,57735.171256,4.38307,102.461610,1570.143349,11463.243457,2.253782e+04,125.662569,161.982694,5.392918,9.362136,...,2.354078,2.481294,1383.885453,1.977911,3.731785,6.622676,10.551631,46.906557,1.96158,6.399923
min,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,71.430000,0.000000,...,0.000000,0.000000,0.000000,1.000000,-1.000000,0.000000,1867.000000,1007.000000,-9.17000,0.000000
25%,49999.750000,1.00000,32.000000,307.000000,1.000000,3.381000e+02,95.000000,54.000000,90.760000,1.000000,...,1.000000,0.000000,365.000000,2.000000,1.000000,2.000000,2012.000000,1950.000000,2.00000,6.000000
50%,99999.500000,5.00000,74.000000,879.000000,3136.000000,4.900000e+02,135.000000,70.000000,90.760000,1.000000,...,1.000000,1.000000,1034.000000,3.000000,2.000000,3.000000,2017.000000,1973.000000,3.39000,9.000000
75%,149999.250000,8.00000,176.000000,2064.000000,13570.250000,7.990000e+02,190.000000,90.000000,90.760000,1.000000,...,3.000000,2.000000,2169.000000,3.000000,3.000000,4.000000,2019.000000,1996.000000,4.57000,13.000000
max,199999.000000,15.00000,418.000000,8225.000000,44189.000000,9.999999e+06,25000.000000,12613.000000,90.760000,2241.000000,...,12.000000,9.000000,7218.000000,560.000000,999.000000,999.000000,2919.000000,2090.000000,12.87000,121.000000


In [4]:
df_train

,ID,区域1,区域2,区域3,街道,上传日期,房屋租金,服务费,供暖费用,电力基础价格,...,邮政编码,房间数量,所处楼层,建筑楼层,有花园,最后翻新年份,是新建筑,建成年份,价格趋势,上传图片数
0,0,0,0,0,0,Oct19,569.0,140.0,NaN,NaN,...,0,3.0,2.0,3.0,False,NaN,False,1971.0,4.62,6
1,1,1,1,1,1,Oct19,560.0,120.0,NaN,NaN,...,1,2.0,NaN,NaN,False,NaN,False,1967.0,5.95,7
2,2,0,2,2,2,Feb20,720.0,NaN,NaN,NaN,...,2,1.0,1.0,1.0,False,NaN,False,2018.0,3.83,10
3,3,2,3,3,3,May19,695.0,153.0,NaN,NaN,...,3,2.0,2.0,3.0,False,NaN,False,1972.0,5.24,7
4,4,3,4,4,1,Oct19,1200.0,350.0,NaN,NaN,...,4,4.5,2.0,2.0,True,NaN,False,NaN,5.63,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,199995,6,71,4704,536,May19,220.0,120.0,NaN,NaN,...,4469,2.0,NaN,NaN,True,NaN,False,1710.0,2.05,1
199996,199996,8,11,157,2531,Feb20,534.0,131.0,NaN,NaN,...,161,1.0,2.0,6.0,False,NaN,False,1969.0,8.36,7
199997,199997,5,9,596,11079,Sep18,225.0,82.0,43.0,90.76,...,625,1.0,0.0,NaN,False,NaN,False,1980.0,5.66,10
199998,199998,1,262,5697,431,Oct19,1142.0,228.0,NaN,NaN,...,5201,3.0,2.0,3.0,False,2017.0,False,NaN,2.92,11


In [5]:
df_test

,ID,区域1,区域2,区域3,街道,上传日期,服务费,供暖费用,电力基础价格,有阳台,...,邮政编码,房间数量,所处楼层,建筑楼层,有花园,最后翻新年份,是新建筑,建成年份,价格趋势,上传图片数
0,200000,5,9,2360,13380,Feb20,250.00,NaN,NaN,True,...,2435,3.0,NaN,NaN,False,2019.0,False,NaN,3.00,5
1,200001,1,175,8202,25,Sep18,130.00,NaN,90.76,True,...,7192,1.0,NaN,NaN,True,NaN,False,2013.0,5.06,6
2,200002,9,307,1883,1,Feb20,200.00,NaN,NaN,True,...,1982,2.0,-1.0,NaN,False,NaN,False,2015.0,3.14,5
3,200003,9,87,123,5819,May19,250.00,NaN,NaN,True,...,125,4.0,0.0,3.0,False,2019.0,False,1969.0,2.57,10
4,200004,13,42,2316,3376,May19,160.00,NaN,NaN,True,...,2390,2.5,NaN,NaN,False,NaN,False,1965.0,2.52,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,249995,12,169,423,50318,Feb20,28.00,38.00,NaN,True,...,439,1.0,1.0,5.0,True,2013.0,False,1983.0,0.19,9
49996,249996,0,0,1845,1363,Oct19,150.00,NaN,NaN,False,...,1945,3.0,NaN,NaN,False,NaN,False,1967.0,3.85,10
49997,249997,6,146,1639,32166,Oct19,245.27,106.45,NaN,True,...,709,4.0,NaN,NaN,False,NaN,True,2019.0,3.68,11
49998,249998,5,9,224,26407,Feb20,272.00,NaN,NaN,True,...,228,4.0,2.0,4.0,False,NaN,False,2015.0,4.24,22


In [6]:
# data = pd.concat([df_train, df_test], axis=0, ignore_index=True)
# data

In [52]:
# data.drop(columns=['ID'], inplace=True)

In [53]:
# data.drop(columns=['上传日期'], inplace=True)

In [54]:
# data

In [55]:
# multi_col_1 = pd.MultiIndex.from_tuples(
#     [('Wind', 'mph'), ('Wind', 'm/s')]
# )
# df_multi_level_1 = pd.DataFrame(
#     [[13, 5.5], [19, 8.5]],
#     index=['London', 'Oxford'],
#     columns=multi_col_1
# )
# df_multi_level_1

In [56]:
# dd = df_multi_level_1.stack()
# dd

In [57]:
# Same as 
# df_multi_level_1.stack(level=-1)
# df_multi_level_1.stack(-1)

In [58]:
# df_single_level

In [59]:
# dup_row = data.duplicated()
# arr = list(dup_row)
# a_count = arr.count('True')
# a_count

In [60]:
# features = [f for f in df_test.columns if f not in ['ID']]
# features

In [61]:
# # df_test.duplicated()
# dup_row = df_test.duplicated(subset=features, keep=False)
# arr = list(dup_row)

In [62]:
# a_count = arr.count('True')
# a_count

In [7]:
# 对缺失值进行填充
df_train.fillna(-1, inplace=True)
df_test.fillna(-1, inplace=True)

df_train.isnull().sum()

ID        0
区域1       0
区域2       0
区域3       0
街道        0
上传日期      0
房屋租金      0
服务费       0
供暖费用      0
电力基础价格    0
有阳台       0
没有停车位     0
有厨房       0
有地窖       0
居住面积      0
房屋状况      0
内饰质量      0
可带宠物      0
加热类型      0
有电梯       0
房屋类型      0
邮政编码      0
房间数量      0
所处楼层      0
建筑楼层      0
有花园       0
最后翻新年份    0
是新建筑      0
建成年份      0
价格趋势      0
上传图片数     0
dtype: int64

In [8]:
df_train.describe()

,ID,区域1,区域2,区域3,街道,房屋租金,服务费,供暖费用,电力基础价格,没有停车位,...,加热类型,房屋类型,邮政编码,房间数量,所处楼层,建筑楼层,最后翻新年份,建成年份,价格趋势,上传图片数
count,200000.000000,200000.00000,200000.000000,200000.000000,200000.000000,2.000000e+05,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,99999.500000,5.11352,112.193435,1452.648445,8684.393950,7.050609e+02,146.809772,23.918151,14.682812,-0.191430,...,2.381500,1.720645,1466.059120,2.638721,1.524785,1.911825,605.151015,1548.523680,3.358908,9.789215
std,57735.171256,4.38307,102.461610,1570.143349,11463.243457,2.253782e+04,126.336451,98.315975,34.240065,5.617407,...,2.354824,2.487229,1383.885453,1.977911,3.573508,5.723826,924.104872,805.732958,1.987137,6.399923
min,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000e+00,-1.000000,-1.000000,-1.000000,-1.000000,...,-1.000000,-1.000000,0.000000,1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-9.170000,0.000000
25%,49999.750000,1.00000,32.000000,307.000000,1.000000,3.381000e+02,90.000000,-1.000000,-1.000000,-1.000000,...,1.000000,0.000000,365.000000,2.000000,0.000000,-1.000000,-1.000000,1900.000000,1.960000,6.000000
50%,99999.500000,5.00000,74.000000,879.000000,3136.000000,4.900000e+02,131.000000,-1.000000,-1.000000,-1.000000,...,1.000000,0.000000,1034.000000,3.000000,1.000000,2.000000,-1.000000,1962.000000,3.370000,9.000000
75%,149999.250000,8.00000,176.000000,2064.000000,13570.250000,7.990000e+02,185.000000,50.000000,-1.000000,1.000000,...,3.000000,2.000000,2169.000000,3.000000,3.000000,4.000000,2008.250000,1992.000000,4.560000,13.000000
max,199999.000000,15.00000,418.000000,8225.000000,44189.000000,9.999999e+06,25000.000000,12613.000000,90.760000,2241.000000,...,12.000000,9.000000,7218.000000,560.000000,999.000000,999.000000,2919.000000,2090.000000,12.870000,121.000000


# 2.特征工程

In [9]:
# 对特征的类型进行处理
# 将上传日期变成年份和月份
upload_date = df_train['上传日期'].values      # 把上传日期拿出来
year_data = []
month_data = []
month_to_num = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 
                'May':5, 'Jun':6, 'Jul':7, 'Aug':8,
                'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12}      # 转换

# 利用正则表达式把年份、月份拿出来
for i in upload_date:
    year_data.append(int(re.findall(r"[0-9]*[.]?[0-9]+", i)[0]))
    month_data.append(month_to_num[re.findall(r"[a-zA-Z]+\’*[a-z]*", i)[0]])

df_train['上传日期_年'] = year_data
df_train['上传日期_月'] = month_data
df_train.drop(columns=['上传日期'], inplace=True)

In [10]:
# 将特征值为布尔值变成数字
features_bool = ['有阳台', '有厨房', '有地窖', '有电梯', '有花园', '是新建筑']
df_train[features_bool] = df_train[features_bool].astype(int)

In [11]:
# 将特征值为多类型的变成数字
pet_dict = {'no':0, "yes":1, "negotiable":2}
pet_num = []
pet_bool = df_train['可带宠物'].values
for pet in pet_bool:
    if pet == -1:
        pet_num.append(-1)
    else:
        pet_num.append(pet_dict[pet])

df_train['可带宠物'] = pet_num

In [12]:
# 对测试集特征的类型进行处理
# 将上传日期变成年份和月份
upload_date = df_test['上传日期'].values
year_data = []
month_data = []
month_to_num = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 
                'May':5, 'Jun':6, 'Jul':7, 'Aug':8,
                'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12}
for i in upload_date:
    year_data.append(int(re.findall(r"[0-9]*[.]?[0-9]+", i)[0]))
    month_data.append(month_to_num[re.findall(r"[a-zA-Z]+\’*[a-z]*", i)[0]])

df_test['上传日期_年'] = year_data
df_test['上传日期_月'] = month_data
df_test.drop(columns=['上传日期'], inplace=True)

# 将特征值为布尔值变成数字
features_bool = ['有阳台', '有厨房', '有地窖', '有电梯', '有花园', '是新建筑']
df_test[features_bool] = df_test[features_bool].astype(int)

pet_dict = {'no':0, "yes":1, "negotiable":2}
pet_num = []
pet_bool = df_test['可带宠物'].values
for pet in pet_bool:
    if pet == -1:
        pet_num.append(-1)
    else:
        pet_num.append(pet_dict[pet])

df_test['可带宠物'] = pet_num

# 3.异常值处理

In [13]:
# 去除异常值
# 拿出 房屋租金==0 的行的索引
index_zero = df_train.loc[df_train['房屋租金'] == 0].index
print("房租价格等于0的数量有{0}".format(len(index_zero)))
df_train.drop(index = index_zero, inplace=True)
df_train = df_train.reset_index(drop=True)

房租价格等于0的数量有68


In [14]:
# 拿出 居住面积=0 的行的索引
# 去除
index_zero_sq = df_train.loc[df_train['居住面积'] == 0].index
print("房租价格等于0的数量有{0}".format(len(index_zero_sq)))
df_train.drop(index = index_zero_sq, inplace=True)
df_train = df_train.reset_index(drop=True)

房租价格等于0的数量有52


In [15]:
# 房屋租金这里的异常值，个人认为4000会比较好，但是由于一开始设置成了3000，所以目前以3000为基准进行测试
index_zero = df_train.loc[df_train['房屋租金'] > 3000].index
print("房租价格等于0的数量有{0}".format(len(index_zero)))
df_train.drop(index = index_zero, inplace=True)
df_train = df_train.reset_index(drop=True)

房租价格等于0的数量有997


In [16]:
# 去除房屋面积>300
index_high = df_train.loc[df_train['居住面积'] > 300].index
print("房租价格超过50的数量有：{0}".format(len(index_high)))
df_train.drop(index = index_high, inplace=True)
df_train = df_train.reset_index(drop=True)

房租价格超过50的数量有：45


In [17]:
# 服务费>750
index_high = df_train.loc[df_train['服务费'] >750].index
print("服务费超过750的数量有：{0}".format(len(index_high)))
df_train.drop(index = index_high, inplace=True)
df_train = df_train.reset_index(drop=True)

服务费超过750的数量有：64


# 4.构建模型训练

In [18]:
# 去除后的特征
features = df_train.columns.drop(['ID','房屋租金'])
# LightGBM 构建数据集
lgb_train = lgb.Dataset(df_train[features], df_train['房屋租金'])

In [19]:
# 参数设置
param = {
    "seed":1111,
    "objective": "regression_l1",
    "learning_rate" : 0.1,  # 学习率
    'num_leaves': 256,  # 叶子节点的数，控制树模型复杂度
    "max_bin":512,  # 较小训练速度更快，较大训练速度慢但准确率可能高
    'bagging_fraction': 0.7,    # 设置bagging，每次迭代时用的数据比例
    'feature_fraction': 0.6,    # 特征子抽样，意味着在每次迭代中随机选择60％的参数来建树
    "metric": "mae",
    "boosting": "gbdt"  # 随机梯度增强树
}

In [86]:
# nfold：10折交叉
# early_stopping_rounds 如果一次验证数据的一个度量在最近的 30 回合中没有提高，模型将停止训练
# num_boost_round：迭代次数
# 开始训练
model = lgb.cv(param, train_set=lgb_train, nfold=10, stratified=False, 
               num_boost_round=200, verbose_eval = 100,early_stopping_rounds=30,
               return_cvbooster=True,)

d:\Users\96212\anaconda3\envs\ML\lib\site-packages\lightgbm\engine.py:577: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] There are no meaningful features, as all feature values are constant.


LightGBMError: Length of label is not same with #data

In [20]:
booster = model['cvbooster']
booster

# 5.模型预测以及输出文件

In [21]:
test_pred = np.zeros(df_test.shape[0])
test_pred

array([0., 0., 0., ..., 0., 0., 0.])

In [22]:
# df_test[features]

In [23]:
# 有几折验证，就有几个模型
# 对预测值进行平均
for model in booster.boosters:
   test_pred += model.predict(df_test[features]) / 10
test_pred

array([ 817.54203037,  431.16576254,  686.20745417, ..., 1422.24928103,
       1255.79287519,  287.76521093])

In [24]:
# 导出csv文件
df_test['房屋租金'] = test_pred
df_test[['ID', '房屋租金']].to_csv('sublgb_16f_31_2000.csv', index=False, encoding='utf-8')